# Get top Video Games through Web Scraping, a raw file and an API.


### ABSTRACT


We are working on a video game dataset and will be extracting video game data from 3 different data sources and then will be munging them together to form a consistent dataset. We will perform several operations over the dataset extracted to make the data clean and error free and consistent. After that we will be developing a database from using the extracted source data and display it in the form of an Entity-Relationship Diagram.
The dataset used is the vgchartz.com. It contains details about Video Game ratings, genres, publisher, year of release, description.

### DATA                                                                                                    
Data gathered is from all the three sources are as follows. Data consists of                                                
Id     

Name     

Description   

Genre

Publisher

Year

Platform

RAWG Score

Critic_Score                                                                      

## Importing Libraries

In [1]:
import requests
import pandas as pd
import json
import os
import rawgpy
from bs4 import BeautifulSoup
import numpy as np

### DATA SOURCE 1: Using Web Scraping using Beautiful Soup

### What is Beautiful Soup?

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.


### The site we are going to use is http://www.vgchartz.com/gamedb/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0. Please visit the link to get information on what is being scrapped.



In [ ]:
# Fetching the tags from the website
url = 'http://www.vgchartz.com/gamedb/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0'
html = requests.get(url, headers = {'User-Agent':'Mozilla/5.0'})
print(html.status_code) 
# Printing the status code, 200 means the request has succeeded

In [ ]:
#Displaying the above format to more readable format using html parser
soup = BeautifulSoup(html.content, 'html.parser')
print(soup.prettify)

In [ ]:
# We will be extracting Rank, Name, Platform and Publisher from vgchartz.com 
k=[] 
rank=[]
gname=[]
publisher=[]
platform=[]
release_date=[]
# Retrieving values using a for loop
for tag in soup.find_all('a'):
    if tag['href'].startswith('http://www.vgchartz.com/game/'):
        k.append(tag.get_text().strip())
        data=tag.parent.parent.find_all('td')
        if data!=[]:
            rank.append(np.int32(data[0].string))
            platform.append(data[3].find('img').attrs['alt'].strip(' '))
            publisher.append(data[4].string.strip(' '))
gname = k[10:] # Our data starts from index position 10 onwards
# Creating a dictionary to store the column names for the dataframe
columns = {
    'Id': rank,
    'Name':gname,
    'Platform':platform,
    'Publisher':publisher
}
df = pd.DataFrame(columns) # Creating a dataframe with column names Rank, Name, Platform and Publisher.
df = df[[
    'Id', 'Name', 'Platform',
    'Publisher']]
# Saving the obtained dataframe on a file named vgsales.csv
df.to_csv("vgsales.csv", sep=",", encoding='utf-8', index=False) # Saves the data to .csv file
#df.drop_duplicates(subset ="Name",keep = False, inplace = True)
df

### Printing the information of the top first rated movie

In [ ]:
gname[0]

In [ ]:
print(df.isnull().any())
print(df.columns)

## DATASOURCE 2 - Using Raw Data

In [ ]:
df2 = pd.read_csv('vgsales2019.csv')
#df2.drop(['Rank','ESRB_Rating','Platform','Publisher','Developer','User_Score', 'Total_Shipped', 'Global_Sales', 'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales'], axis=1, inplace= True)

### Displaying the output

In [ ]:
df2.head()

### Checking the information of the data - data type and total number of records in each column

In [ ]:
df2.info()

### How to find the missing values

In [ ]:
# checking missing, NaN data in the dataframe through CSV
df2.isnull().any()

### Checking the total null values in the column using sum() function

In [ ]:
df2.isnull().sum()

### Checking the shape of the data

In [ ]:
df2.shape

### Checking the columns present in the data

In [ ]:
df2.columns

In [ ]:
df2.drop(['Rank'],axis=1, inplace= True)

In [ ]:
# Merging the two dataframe df and dframe having unique names
gameData = pd.merge(df,df2, 
                 on = 'Name')

In [ ]:
gameData

In [ ]:
gameData.to_csv("GameData.csv",encoding="utf-8",index=False)

In [ ]:
gameData.drop_duplicates(subset ="Name",keep = "first", inplace = True)
gameData

In [ ]:
print(len(gameData)) 

## DATASOURCE 3 - Using API

#### What is an API:

API stands for Application Programming Interface, and it lets developers integrate any two parts of an application or any different applications together. It consists of various elements such as functions, protocols, and tools that allow developers to build applications. A common goal of all types of APIs is to accelerate the development of applications by providing a part of its functionality out-of-the-box, so developers do not have to implement it themselves.

We will be using an API Wrapper to get data from the video game database www.rawg.io. Since it's a public database we won't be requiring an API key for using www.rawg.io's API.
We will be importing the API wrapper python class for www.rawg.io, which is rawgpy.
https://rawgpy.readthedocs.io/en/latest/ https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

### Importing the libraries rawgpy 

In [ ]:
import rawgpy 

#### We will be using the data obtained from the Web Scraping to search through rawgpy.io to get ratings and description of the game. The ratings will be on a scale of 5. 

In [ ]:
# Storing the names of the game on gname
gname = gameData['Name']
# First initiate two empty list for storing the raw data
rawg_ratings = []
description = []

In [ ]:
rawg = rawgpy.RAWG("User-Agent, this should identify your app")
for name in gname:
    results = rawg.search(name)  # defaults to returning the top 5 results
    game = results[0] # selects the first result of the search
    game.populate() # get additional information on the game
    rawg_ratings.append(game.rating)
# Since description has <p> and </p>, we will replace it with ""
    desc = game.description.replace("</p>","")
    description.append(desc.replace("<p>",""))
print (len(rawg_ratings))
print(len(description))
# It will take 2-3 minutes to complete the iteration

In [ ]:
gameData["RAWG Score"] = rawg_ratings
gameData["Description"] = description

In [ ]:
gameData

### Cleaning and auditing the data in gameData

### How to find missing values

In [ ]:
# checking missing, NaN data in the dataframe 
gameData.isnull().any()

### Checking the total null values in the column using sum() function

In [ ]:
gameData.isnull().sum()

### Dropping all columns which have large numbers of missing values

In [ ]:
gameData.drop(['ESRB_Rating','Platform_x', 'Publisher_x','Developer',
          'User_Score', 'Total_Shipped', 'Global_Sales', 'NA_Sales', 
          'PAL_Sales', 'JP_Sales', 'Other_Sales'], axis=1, inplace= True)

### Saving the dataframe to a temporary file called temp.csv
The program isn't able to detect any null values in description attribute. So, we well save the dataset to a temp.csv file and then we will importing it and will be saving it on gameData

In [ ]:
gameData.to_csv('temp.csv', encoding = 'utf-8', index = False)
# Storing the file in gameData
gameData = pd.read_csv('temp.csv') 
# Checking the total number of null values
gameData.isnull().sum()

### Removing all records having null values

In [ ]:
gameData.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
# Checking the null values
gameData.isnull().sum()

In [ ]:
gameData.head()


### Checking shape of the data

In [ ]:
gameData.shape

### Checking the entities present in the database

In [ ]:
gameData.columns

### Checking that attribute Rank is unique or not 

In [ ]:
gameData['Id'].is_unique

### Checking the information of the data - data type and total number of records in each column

In [ ]:
gameData.info()

## Technique to use visualization to understand the data better

Here we are using seaborn, matplotlib and scipy to analyse our dataset

In [ ]:
# Importing liabries used for visualization

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### We use the Critic_Score and RAWG Score from our dataframe to illustrate how the vote count is distributed in the data set

In [ ]:
critic = gameData['Critic_Score']

sns.distplot(critic)

### From the result we can see that the Critic Score is not normally distributed. Most of the scores lie between 8 and 9.5.

In [ ]:
rawg_score = gameData['RAWG Score']
sns.distplot(rawg_score)

### Here also, the we can see that RAWG Score is not normally distributed. Most of the games in the dataframe have a score of 4.5.

In [ ]:
sns.boxplot(np.array(gameData['Critic_Score']))

In [ ]:
sns.boxplot(np.array(gameData['RAWG Score']))

## CONCEPTUAL MODEL

## Generating Tables by reformatting data

In [ ]:
# Created a Game_Ratings table using the below columns
Game_Ratings = gameData.loc[:,['Id', 'RAWG Score', 'Critic_Score']]
Game_Ratings.head()

In [ ]:
# Created a table platform details 
columns = {'Id': gameData['Id'],
          'Platform': gameData['Platform_y']}
Platform = pd.DataFrame(columns)
Platform.head()

In [ ]:
# Created a table Game Details
columns = {'Id': gameData['Id'],'Name': gameData['Name'],'Description': 
               gameData["Description"],'Genre': 
               gameData["Genre"], 'Publisher': gameData['Publisher_y'],
           'Year':gameData['Year']}
Game_Details = pd.DataFrame(columns)
Game_Details.head()

### Generating CSV files (Exporting the files to the PC)

In [ ]:
Game_Ratings.to_csv('Game_Ratings.csv', encoding = 'utf-8', index = False)

In [ ]:
game_ratings = pd.read_csv('Game_Ratings.csv')

In [ ]:
game_ratings.head()

In [ ]:
Game_Details.to_csv("Game_Details.csv", encoding = 'utf-8', index = False)
game_details_ = pd.read_csv('Game_Details.csv')
game_details_.head()

In [ ]:
Platform.to_csv("Platform.csv", encoding = 'utf-8', index = False)
platform = pd.read_csv('Platform.csv')
platform.head()

In [ ]:
game_details = pd.merge(game_details_, 
                     platform,on = 'Id')
game_details

### Combining the two datasets over a common key(id) and the data remains consistent

In [ ]:
game_details_combined = pd.merge(game_details_, 
                                 game_ratings,on='Id')
game_details_combined

### ER MODEL

In [ ]:
from IPython.display import Image
Image('VideoGameERD.png')

The above figure shows us an entity-relationship model of three different tables from the following: data from game title table,data from ratings table, data from platform table and data from other details. We can infer that id acts as a primary key, meaning that all the tables are linked with game_title. The fields title,overview,release_date,original_title
from movie_title. id holds the common data in movietitle and language. Hence, we can conclude that on merging the three tables we obtain one table which defines a combined data set of values from different places yet relatable, which is known as a conceptual database schema.

### AUDIT VALIDITY/ACCURACY

We say data is accurate only when it is neat and with no null or junk values. By using drop function, all the unwanted null values were deleted from the above rows and columns which gives a report on valid and accurate data. We have also dropped some of the columns which had large number of null values.

### AUDIT COMPLETNESS

In real world, when a list of video games from a particular customer is requested, a list of it will be displayed or presented, similarly when we compare it with above data too, we get proper real time data showing correct information for all the video games.  


### AUDIT CONSISTENCY/UNIFORMITY

The datasets which have been used in this assignment show a uniform relationship between each of the dataset since they are linked to each other by a common attribute. 

### REPORT

fies used : vgsales2019.csv                                                                         
files genearted:  Platform.csv, Game_Ratings.csv,Game_Details.csv                                      
Data is reformatted to fit into a conceptual model. Data gathered from different sources Web API, Web scraping, Raw file and are mergerd together to fit into a conceptual model.                                              

Code used:                                                                                                 
Step 1. Extraction of Data                                                                                    
3 main methods were used for the extraction of data:                                                        
1. Using the API Wrapper:                                                                                             
Here since the rawg.io is a public video game database, we won't be neeeding an API key. We will be using an API Wrapper to get the RAWG Score of each game we scraped from the website vgsales.com. We will be importing the Python API wrapper class rawgpy so as to do the same process as an API.                                                                    
pandas to create data frames from the raw data                                                                 
2. Using the website to scrap the data                                                                       
Here the data was extracted using the sites data directly using the libraries like:                               
request to access the website using the URL (http://www.vgchartz.com/gamedb/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0)                       
BeautifulSoup to scrape the contents of the website                                                          
 find_all() and parent methods were used to find the desired content in the system                              
3. By loading the csv file:                                                                                  
Here the data was extracted using a csv file on the system using the libraries like:                            
Pandas to a read the csv file and load it into data frames                                                    
read_csv method is used to read.csv file                                                                   
Step 2. Cleaning and Auditing Data                                                                             
To gain knowledge about the dataset we used various methods like                                                  
describe, isnull, any, shape, columns, is_unique, info, iloc, loc, os


Code used for Merge  1                                                                                         
game_details = pd.merge(game_details, 
                     platform,on = 'Id')                                             
                                                                                                 
Code used for final merge                                                                                    
game_details_combined = pd.merge(game_details_, 
                                 game_ratings,on='Id')


### CONCLUSION                                                                                                                                                                   
Primary focus of this assignment is to learn how to get the data from different sources, cleaning of data, checking null values present in the data, data munging and to reformat the data to fit a conceptual database model.

### CONTRIBUTION
###### Your contribution towards project. How much code did you write and how much you took from other site or some other source.                                                                            
Ashwin John Chempolil: 15% 

Crispin Sujith Cletus: 10%

By External source: 50%                                                                                        

Provided by the professor : 25% 


### CITATIONS
##### Sources from where you have gained knowledge or used codes, data. It may include Web links, github links, code taken from somewhere etc.
http://www.vgchartz.com/gamedb/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0                                                              
https://www.crummy.com/software/BeautifulSoup/bs4/doc/           https://github.com/GregorUT/vgchartzScrape/blob/master/vgchartzfull.py                        
https://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html
https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-index-labels/
https://rawgpy.readthedocs.io/en/latest/
https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

### LICENSE
Copyright 2019 Ashwin John Chempolil, Crispin Sujith Cletus

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software without restriction, including without limitation the 
rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit
persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the 
Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE 
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR 
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR 
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.